ライブラリインポート

In [1]:
import pandas as pd
from tqdm import tqdm
import os
import datetime

pickleファイルからロード

In [2]:
#pickleデータロード
dfRaceResult = pd.read_pickle('./data/race_result.pkl')
dfRaceInfo = pd.read_pickle('./data/race_Info.pkl')
dfHorseResult = pd.read_pickle('./data/horse_result.pkl')
dfHorsePed = pd.read_pickle('./data/horse_ped.pkl')
#レース結果と競馬戦績にレース情報を結合
dfRaceResulti = pd.merge(left=dfRaceResult,right=dfRaceInfo,
                        how='left', on='raceId',suffixes=['','_i'])
dfHorseResulti = pd.merge(left=dfHorseResult,right=dfRaceInfo,
                        how='left', on='raceId',suffixes=['','_i'])
#戦績テーブルからゴミを除外
df = dfHorseResulti.copy()
columns = []
for cn in df.columns:
    if '_i' not in cn:
        columns.append(cn)
dfHorseResulti = df[columns]


FileNotFoundError: [Errno 2] No such file or directory: './data/horse_result.pkl'

レース結果テーブルに成績を結合

In [ ]:
#空のデータフレームを生成
dfRaceResultH = pd.DataFrame(columns=['horseId','日付'])
#pickleファイルの存在チェック
if(os.path.isfile('./data/race_result_add_record.pkl')):
    #読み込む
    dfRaceResultH = pd.read_pickle('./data/race_result_add_record.pkl')

#レースを日付降順にソートする
dfHorseResulti = dfHorseResulti.sort_values(('日付'),ascending=False)

indexList = dfRaceResulti.index
escapeList = dfRaceResultH.index
#一時撤退用データフレームを初期化
saveDf = pd.DataFrame(columns=dfRaceResultH.columns)

#レース結果のテーブルを1行ずつ取り出す
for cnt, rowIndex in tqdm(enumerate(indexList), total = len(indexList)):
    #既に処理済みのレコードだったらスキップ
    if rowIndex in escapeList:
        continue
    try:
        #レース結果から1行分のデータを取り出す
        rowDf = dfRaceResulti.loc[dfRaceResulti.index==rowIndex]
        #horseIdと日付を取り出す
        horseId = rowDf['horseId'].iloc[0]
        dt = rowDf['日付'].iloc[0]

        #horseIdで絞込レース結果の日付以来の直近9レース分をループ処理実行
        for idx,tmpDf in \
            enumerate(dfHorseResulti[(dfHorseResulti['HorseId']==horseId)&\
                                     (dfHorseResulti['日付']<dt)].head(9).iterrows()):
            idxx = idx+1
            tmpDfx = pd.DataFrame(tmpDf[1]).T
            #列名にループ回数でサフィックスを付加
            tmpDfx.columns = tmpDfx.columns + '_' + str(idxx)
            #結合の為にidexを合わせる
            tmpDfx.index = rowDf.index
            #結合の実行
            rowDf = pd.concat([saveDf, rowDf], axis=0)
        #1行分のレコードを一時テーブルに結合
        saveDf = pd.concat([saveDf, rowDf], axis=0)
        #2000回に一度本体のテーブルに結合して一時テーブルを空にする
        if cnt % 2000 == 0:
            dfRaceResultH = pd.concat([dfRaceResultH, saveDf], axis=0)
            saveDf = pd.DataFrame(columns=dfRaceResultH.columns)
    except:
        break
dfRaceResultH = pd.concat([dfRaceResultH, saveDf], axis=0)
dfRaceResultH.to_pickle('./data/race_result_add_record.pkl')

出走間隔を計算

In [ ]:
df = pd.read_pickle('./data/race_result_add_record.pkl')
#スカラー関数の定義
def CalcInterval(x):
    try:
        interval = datetime.datetime.strptime(x['日付'], '%Y/%m/%d')-\
                    datetime.datetime.strptime(x['日付_1'], '%Y/%m/%d')
        return interval.days
    except:
        return 0
#apply関数で一括処理
df['出走間隔'] = df.apply(CalcInterval, axis=1)

レース結果に血統データを結合

In [ ]:
dfRaceResultP = pd.merge(left=df, right=dfHorsePed, how='left', on='horseId')
dfRaceResultP.to_pickle('.data/dataset.pkl')